# Tutorial 9 with Solutions

In this Tutorial we begin by designing and implementing functions for enciphering and deciphering messages with one of the original (but now highly insecure) cipher systems, the Caesar Cipher. We then see how to reimplement the RSA protocol for signature/authentication of messages. Finally, in a bonus question, that you can complete in your own time, we look at another instance of encoding and decoding of data in Python. (This time instead of strings/texts we encode finite sets of integers.) 

### Extra resources

You should make sure that the file  `cryptography_functions.py` is in your present working directory. This file contains functions from the lectures or Weeks 8-9. 


## Question 1: the Caesar cipher 

Suppose that we have an alphabet of letters/characters in which to write a message. For example suppose that our alphabet consists of the 27 lower case letters of the our usual alphabet and the space character `' '`. In Python we can think of the alphabet as  a string of characters and, of course assign it a name 
```python
ALPH_1 = 'abcdefghijklmnopqrstuvwxyz '
``` 
so that every letter is associated with its index in ALPH_1. For example `0` and `4` correspond to `ALPH_1[0]` which is `a` and `ALPH_1[4]` which is `e`. 

Now we want to be able to encipher any message using our alphabet of 27 letters by a circular shift which we can choose to be any integer in the interval $[0,26]$. For example, suppose that we choose shift 3, then the enciphering process corresponds to `'a'` $\mapsto$ `'d'`, `'b'` $\mapsto$ `'e'`, $\dots$ , `'x'` $\mapsto$ `' '`, `'y'` $\mapsto$ `'a'`, `'z'` $\mapsto$ `'b'`, `' '` $\mapsto$ `'c'`. Then, using shift 3 the message 
```python
'go left'
``` 
is enciphered as 
```python
'jrcohiw'
```
which, at least at first glance, looks like nonsense to anyone who does not know how we have enciphered the message. Note however, that if you know that shift then it is easy to decipher the message.  

According to folklore Romans in the time of Julius Caesar used this method to encipher messages before sending them. The sender and the receiver would both know the shift used and, given the state of Mathematical knowledge at the time,  anyone intercepting the message would have very no idea of how to decipher it. (With modern tools this is very easy...) 

Your task in this question is to design functions that encipher and decipher messages using the **Caesar cipher** method just described. To do this you will first define functions that perform the shift map and its inverse on individual characters. We begin by defining two different alphabets for use by our functions. 


In [ ]:
# DEFINITION OF THE FIRST ALPHABET 
import string
ALPH_1 = string.ascii_lowercase + " "
print("ALPH_1 is of length: m = {}".format(len(ALPH_1)))
print("It is defined to be the string below.")
print("Note that the last character is the space character.")
ALPH_1

In [ ]:
# DEFINITION OF THE SECOND ALPHABET 
ALPH_2 = string.ascii_letters + ",.;:?! " 
print("ALPH_2 is of length: m = {}".format(len(ALPH_2)))
print("It is defined to be the string below.")
print("Note that the last character is the space character.")
ALPH_2

### Part 1: the Caesar shift map

Design a function of the form `caesar_shift(letter,shift,alphabet)` where `letter` is a character (i.e. a 1 character string), `shift` is a non-negative integer, and `alphabet` is a string alphabet such as `ALPH_1` above. Your function should perform the circular shift map as describe above to the character `letter`. Thus (using the example above with `shift` being `3`) you should find that `caesar_shift(`'a'`,3,ALPH_1)` returns `'d'`, `caesar_shift(`'y'`,3,ALPH_1)`
returns `'a'` and `caesar_shift(`' '`,3,ALPH_1)` returns `'c'`. 

**Note 1.** You will need to work modulo `len(alphabet)`.

**Note 2.**  We usually choose `shift` to be less than the number of letters in our alphabet (i.e. `len(alphabet)`). However since we work modulo `len(alphabet)` we could in fact allow `shift` to be *any* non-negative integer.

In [ ]:
def caesar_shift(letter,shift,alphabet): 
    # YOUR CODE HERE
    size = len(alphabet)                # Number of letters/characters in the input alphabet
    index = alphabet.find(letter)       # index now gives the place of letter (starting at 0) in the alphabet
    new_index = (index + shift) % size  # Perform the shift map to get the index of the new letter
    new_letter = alphabet[new_index]    # Extract this new letter
    return new_letter

In [ ]:
# TESTING AREA : USE THIS CELL TO CHECK OR TEST YOUR CODE
# A COUPLE OF PRELIMINARY CHECKS...
print(caesar_shift('a',3,ALPH_1))
print(caesar_shift('y',3,ALPH_1))
print(caesar_shift(' ',3,ALPH_1))
assert caesar_shift('t',13,ALPH_1) == 'f'
caesar_shift(',',8,ALPH_2) == 'b'

### Part 2: inverting the Caesar shift map

Design a function of the form `caesar_invert_shift(letter,shift,alphabet)` where `letter` is a character, `shift` is a non-negative integer, and `alphabet` is a string alphabet such as `ALPH_1` above. Your function should perform the inverse map of the  circular shift map to the character `letter`. Thus (using the example above with `shift` being `3`) you should find that `caesar_shift('d',3,ALPH_1)` returns `'a'`, `caesar_shift('a',3,ALPH_1)`
returns `'y'` and `caesar_shift('c',3,ALPH_1)` returns `' '`. 

In [ ]:
def caesar_invert_shift(letter,shift,alphabet): 
    # YOUR CODE HERE 
    size = len(alphabet)
    # Compute the value inverse_shift needed to invert the shift.
    inverse_shift = (size - shift) % size  
    # And now simply use caesar_shift with this value. 
    return caesar_shift(letter,inverse_shift,alphabet)

In [ ]:
caesar_invert_shift('e',3,ALPH_1)

### Part 3: a table to test your Caesar shift functions

At this point in the development process it is good practice to thoroughly test the functions that you have just designed (as these are fundamental to the enciphering and deciphering algorithms below). To do this you should now design a function  `caesar_table` taking as input a string `alphabet`. Your function should guess a non-negative integer `shift` that is less than `len(alphabet)`, print out this number, and then print out a simple table of three columns containing a row for each character in `alphabet`. In the first column the character `letter` (say)  should appear, in the second column the letter `enc_letter` (say) returned by `caesar_shift(letter, shift, alphabet)` should appear, and in the third column the letter `dec_letter` (say) returned by `caesar_shift(enc_letter, shift, alphabet)` should appear. For example the first 8 lines output by
```python
caesar_table(ALPH_1): 
```
should look like this: 
```
The shift is 10 

a k a
b l b
c m c
d n d
e o e
f p f
```
or similar (depending on the random shift generated - here it is 10). 

In [ ]:
from random import randrange
def caesar_table(alphabet):
    # YOUR CODE HERE
    size = len(alphabet)
    # Guess a number in  {0,1,...,size-1} for the shift
    shift = randrange(size)
    print("The shift is",shift,"\n")
    # For each letter in the alphabet print out a row containing the letter,
    # its enciphered form (as a letter) and its deciphered form. 
    for letter in alphabet:
        enc_letter = caesar_shift(letter,shift,alphabet)
        dec_letter = caesar_invert_shift(enc_letter,shift,alphabet)
        print(letter,enc_letter,dec_letter)
    return None

In [ ]:
# Run this cell repeatedly to see tables using random shift values
# Replace ALPH_1 by ALPH_2 to see the same for the second alphabet
caesar_table(ALPH_1) 

### Part 4: enciphering messages using the Caesar cipher

Design a function of the form `caesar_encipher(message,shift,alphabet)` where `message` is a text in the form of a string, `shift` is a non-negative integer and `alphabet` is a string representing an alphabet. Your function should return  the string representing the enciphered message, i.e. the string of the same length as `message` in which  every character `letter` (say)  in `message` has been replaced by the result of `caesar_shift(letter,shift,alphabet)`.   

In [ ]:
def caesar_encipher(message,shift,alphabet): 
    # YOUR CODE HERE
    enc_message = ""
    # Append each enciphered character to enc_message
    for letter in message: 
        enc_message += caesar_shift(letter,shift,alphabet)
    # Alternatively instead of the for loop, use the following line (uncommented) 
    #enc_message = "".join(caesar_shift(letter,shift,alphabet) for letter in message)
    return enc_message

In [ ]:
# Messages for testing 
message1 = "almost none of the writings of democritus survive some survived into the middle ages but they are lost now"
message2 = "Almost none of the writings of Democritus survive. Some survived into the Middle Ages, but they are lost now." 

In [ ]:
# TESTING AREA : USE THIS CELL TO CHECK OR TEST YOUR CODE

### Part 5: deciphering messages using the Caesar cipher

Design a function of the form `caesar_decipher(message,shift,alphabet)` where `message` is an (enciphered) text in the form of a string, `shift` is a non-negative integer and `alphabet` is a string representing an alphabet. Your function should return  the string representing the deciphered message, i.e. the string of the same length as `message` in which  every character `letter` (say)  in `message` has been replaced by the result of `caesar_invert_shift(letter,shift,alphabet)`.   

In [ ]:
def caesar_decipher(message,shift,alphabet): 
    # YOUR CODE HERE
    dec_message = ""
    # Append each deciphered character to dec_message
    for letter in message: 
        dec_message += caesar_invert_shift(letter,shift,alphabet)
    # Alternatively instead of the for loop, use the following line (uncommented)
    # dec_message = "".join(caesar_invert_shift(letter,shift,alphabet) for letter in message)
    return dec_message 

In [ ]:
# HERE'S SOME TESTING USING message1 WITH ALPH_1
# YOU SHOULD REPEATEDLY RUN THIS CELL 
shift = randrange(len(ALPH_1))
print("The shift is", shift)
print("Original message:")
print(message1)
print("Enciphered message: ")
enc_message1 = caesar_encipher(message1,shift,ALPH_1)
print(enc_message1)
print("Deciphered message: ")
dec_message1 = caesar_decipher(enc_message1,shift,ALPH_1)
print(dec_message1)

In [ ]:
# CARRY OUT SIMILAR TESTING FOR message2 WITH ALPH_2
# YOUR CODE HERE 
shift = randrange(len(ALPH_2))
enc_message2 = caesar_encipher(message2,shift,ALPH_2)
print(enc_message2)
dec_message2 = caesar_decipher(enc_message2,shift,ALPH_2)
print(dec_message2)

## Question 2: RSA for message authentication

A common use of the **RSA**  protocol is to digitally sign a message. Suppose that $(p,q)$ and $(N,e)$ are
  the private and public keys computed by Alice as seen in lectures (so $p$, $q$ are $512$-bit primes etc). Then
  Alice **hashes**  her message into a $512$ bit integer $h$.  Alice computes signature $s = h^f$ modulo $N$ where $f$ is the multiplicative inverse of $e$ modulo $\phi(N)$ $-$ the **totient** of $N$ $-$ and sends $s$ to Bob with her message. 
  On receiving the message Bob also hashes the message using the same
  hash function to obtain an integer $h'$. Bob also computes $s^e \equiv h^{fe} \equiv h$ modulo $N$.
  If Bob finds that $h$ and $h'$ are equal he knows that Alice sent the message. I.e$.$ the message has been
  authenticated.
  
**Note.** (Note first that the  above is a simplified description of hashing as are also the following remarks.) Notice that for hashing we choose a fixed bit length. Here the bit length is $512$. Then, given any message we hash it to an integer $h$ of this fixed bit length. If our hash function is good the result $h$ will be an integer whose binary representation is some nonsense sequence of bits. In other words the hash will be highly **irreversible** in the sense that the message cannot be deduced from the hash. Moreover  hash functions are not injective (since the number of possible messages (possibly infinitely many!)  that can be hashed is larger than the number of integers of bit length $512$. Again if our hash function is good it will **avoid collisions** among similar (but distinct) messages (e.g. of the same length). (By "collision" of two or more messages we mean that they are  hashed to the same integer.)  

Your task in this question is to design a function for Alice to sign messages as also a function for Bob to verify the authenticity of a message. 

We will need a certain number of functions from  Week 8 and  from Lecture 9.3. These can be found in the module `cryptography_functions` in the file `cryptography_functions.py`. 

In [1]:
from cryptography_functions import * 

We can check which functions we have imported using help. 

In [ ]:
import cryptography_functions
help(cryptography_functions)

We need a hash function, defined below as `hash_to_512`. (Don't worry about the definition. However we will check its output!) 

In [13]:
from hashlib import sha512

def hash_to_512(message): 
    '''
    Input parameter: message of type string. 
    Returns: message hashed as 512 bit integer.
    '''
    h_bytes = sha512(message.encode('utf-8')).digest()
    h_integer = int.from_bytes(h_bytes,"big")
    return h_integer 

In [15]:
help(hash_to_512)

Help on function hash_to_512 in module __main__:

hash_to_512(message)
    Input parameter: message of type string. 
    Returns: message hashed as 512 bit integer.



Let's check that our `hash_to_512` does indeed output a $512$ bit (more or less at least!) integer on messages of different length. (We can assume that our function is "good" in the sense given above, by definition  of the `sha512` function.) 

In [16]:
message1 = "God plays dice with the universe"
message2 = """The real trouble in quantum mechanics is not that the future trajectory of a particle is 
indeterministic - it's the fact that the past trajectory is also indeterministic! Or more
accurately, the very notion of a 'trajectory' is undefined, since until you measure, there's
just an evolving wave function.""".replace('\n', ' ')

hash1 = hash_to_512(message1)
hash2 = hash_to_512(message2)

print("hash1 in binary has", len(bin(hash1)[2:]), "bits and is:")
print(bin(hash1)[2:])
print("hash1 in decimal is:")
print(hash1)
print("\nhash2 in binary has", len(bin(hash2)[2:]), "bits and is:")
print(bin(hash2)[2:])
print("hash2 in decimal is:")
print(hash2)

hash1 in binary has 511 bits and is:
1011000010010001000110110010000001001101100110000011101001101100001110101011001110111110101010011001001111110000110100100001011101101001100111100001000100011101101101011010001011101100000000010101000001001100000010011101111000000101010100000101001101010011100000001010111010100000101000101011010110011111101101111001010111010010001001110010000001110100111011010111111111010001010110110111110000110111101111011111110000111111001001111110101011110000100101101110100110101000101010110011111000111001010001011110110
hash1 in decimal is:
4623777366293871185710569166472694572972446252431881673341093069074424715628397970285346738937403060035025068413120135813186019740010520194303357555090166

hash2 in binary has 510 bits and is:
111001011100000011101111111100100101100100111101101100101001000101110011010010110010110100011100110100100100011000110100100010111000010101010011011110100110100001110001101100010001100110000011101010010011101110000100011000111111001010010

### Part 1: signing a message in RSA

Design a function of the form `sign(message,p,q,N,e)` for Alice to carry out the RSA signature protocol described above, based on private key $(p,q)$ and public key $(N,e)$. Your function should return the signature $s$ stipulated in the description above. 

**Hint.** Use `hash_to_512` and adapt the code from the definition of `rsa_decrypt` from Lecture 9.3 (also in the file `cryptography_functions.py`. 

In [5]:
def sign(message,p,q,N,e): 
    # YOUR CODE HERE
    h = hash_to_512(message)
    totient = N - (p + q) + 1       # This is (p-1)*(q-1)
    f = modular_inverse(e,totient)  # Note: f * e = 1 (mod totient)
    signature = pow(h,f,N)          # This is h**f (mod N)
    return signature

### Part 2: authenticating a message in RSA

Design a function of the form `authenticate(message,N,e)` for Bob to carry out authentication of `message` using the RSA signature protocol, based on private key $(p,q)$ and public key $(N,e)$. Your function should return `True` (message authenticate) or `False`. 

**Hint.** Use `hash_to_512` and adapt the code from the definition of `rsa_encrypt` from Lecture 9.3 (also in the file `cryptography_functions.py`. 

In [6]:
def authenticate(message, signature, N, e): 
    # YOUR CODE HERE
    h_received = hash_to_512(message) # This is the hash h' of the received message
    h = pow(signature,e,N)            # Note that  signature**e (mod N) = h**(f*e) (mod N) = h 
    return h_received == h            # Only return True if hash of received message is = hash verified by signature

### Part 3: testing your RSA signature/authentication functions

We can now test the system. 

Firstly Alice needs to generate a private key (that only she knows) and a public key (that everybody can know).  Use the imported functionsw `rsa_private_key` and `rsa_public_key`). 

In [7]:
# YOUR CODE HERE (JUST 2 LINES)
(p,q) = rsa_private_key(512)
(N,e) = rsa_public_key(p,q)

Alice now signs both `message1` and `message2` using the RSA protocol based on private key $(p,q)$ and public key $(N,e)$ (so generating `signature1` and `signature2` (say). 

In [8]:
# YOUR CODE HERE (JUST 2 LINES)
signature1 = sign(message1,p,q,N,e)
signature2 = sign(message2,p,q,N,e)

Alice now sends each messsage with its digital signature to Bob. (This is for testing purposes. The point is that Bob should be able to authenticate a message using the correct signature. If he uses the wrong signature the message will not be authenticated.) You should follow the instructions in the following cells.

In [9]:
# Bob authenticates message1 using signature1 and the public key
# YOUR CODE HERE (JUST ONE LINE)
authenticate(message1,signature1,N,e)

True

In [10]:
# To check: Bob's authentication using message2 and signature 1 and the public key fails
# This is the correct behaviour - as the signature is wrong.
# YOUR CODE HERE (JUST ONE LINE)
authenticate(message2,signature1,N,e)

False

In [11]:
# Bob authenticates message2 using signature2 and the public key
# YOUR CODE HERE (JUST ONE LINE)
authenticate(message2,signature2,N,e)

True

In [17]:
# To check: Bob's authentication using message1 and signature 2 and the public key fails
# This is the correct behaviour - as the signature is wrong.
# YOUR CODE HERE (JUST ONE LINE)
authenticate(message1,signature2,N,e)

False

## Bonus Question 3: encoding and decoding finite sets 

In Lecture 9.2  you saw how to encode and decode text messages (i.e. strings in python) as integers. Similar methods have a long history in the study of the algorithmic content of mathematics. For example, a core ingredient in the proofs of Godel's incompleteness theorems published in 1931 - which showed the inherent limitations of any effectively enumerated axiomatic system which is strong enough to model basic arithmetic - was the ability to encode formulae, sentences and proofs of axiomatic systems as integers. Likewise Turing's development of a universal Turing machine in 1936 relied on the fact that every Turing program - which you can think of as just a Python program in our present context - can be uniquely encoded (and so also decoded) as an integer. A standard way to define such coding methods is to firstly define a computable bijective pairing function 
$p : \mathbb{N} \times \mathbb{N} \rightarrow \mathbb{N}$ and then derive from this a computable bijective function that maps sequences (of integers) to integers. 

Closely related to the problem of coding sequences  is that of coding finite sets of integers.
A standard way of doing this is using the bijective function 

$$
f : \{D \,:\, D \subset \mathbb{N} \;\&\; D \;\mbox{is finite}\}  \rightarrow \mathbb{N}
$$

defined such that 

$$
   f(D) =                                                                                     
        \begin{cases}                                                                         
        0              &\mbox{if } D = \emptyset \\                                                             
        \sum_{n \in D} 2^n   &\mbox{otherwise.}
        \end{cases}  
$$

For example if $D = \{0,3,6,7,12\}$ then $f(D) = 2^0 + 2^3 + 2^6 + 2^7 + 2^{12} = 4297$. 

**Note.** You will use the `set` class  to define finite sets in python. Before attempting this exercise you should quickly check out the Appendix below which contains a brief introduction to this class.

**(a)** Write down a function `encode_set` which, on input `D`, where `D` is a finite set, returns the integer code of `D` corresponding to  $f(D)$.

In [ ]:
def encode_set(D):
    # YOUR CODE HERE
    # The code of the empty set is 0
    if len(D) == 0: 
        return 0
    # Otherwise the code of D is the sum of the numbers 2^m 
    # such that m belongs to D
    code_of_D = 0
    for m in D: 
        code_of_D += pow(2,m)
    return code_of_D

In [ ]:
# TESTING AREA : USE THIS CELL TO CHECK OR TEST YOUR CODE

In [ ]:
# Some extra tests 
A = set()
B = {0,1,3,4,6}
C = {1, 2, 4, 5, 6, 7, 8, 12}
assert encode_set(A) == 0
assert encode_set(B) == 91
assert encode_set(C) == 4598

**(b)** Write down a function `decode_to_set` which, given a non-negative integer `n` as input, returns the finite set `D` of which `n` is the encoding under $f$ (i.e. $D$ is such 
that $f(D) = n$). 

**Hint.** There are two cases $n = 0$ and $n > 0$. The first case is easy to deal with. For the latter case let's consider the example given by $n = 91$. Then note that 
$n = 2^0 + 2^1 + 2^3 + 2^4 +  2^6$ and so $n$ encodes the set $ D := \{0,1,3,4,6\}$.  
Now simply rewrite the previous sum (of powers of two) representation of $n$ as follows: 
$$ 
n = 1 \cdot 2^6 + 0 \cdot 2^5 + 1 \cdot 2^4 + 1 \cdot 2^3 
    + 0 \cdot 2^2 + 1 \cdot 2^1 + 1 \cdot 2^0 \,. 
$$
But this means that in python we can extract the set `D` by processing the binary representation `'1011011'` of `n` from the back to the front!

In [ ]:
def decode_to_set(n):
    # YOUR CODE HERE
    the_set = set()
    # 0 is code for the empty set 'set()'
    if n == 0: 
        return the_set
    # Otherwise convert n into binary... 
    binary_n = bin(n)[2:]
    length_n = len(binary_n)
    # And process this from the back to the front so that, for each '1' 
    # in binary_n, the (integer) distance of this '1' from the last digit 
    # (corresponding to the current value of dist) is added to the_set
    for dist in range(0,length_n):
        index = length_n - 1 - dist
        if binary_n[index] == '1': 
            the_set.add(dist)
    return the_set

In [ ]:
# TESTING AREA : USE THIS CELL TO CHECK OR TEST YOUR CODE

In [ ]:
# One friendly test 
# Note that {} is the empty dictionary not the empty set (which is set())
n = 91
assert decode_to_set(n) == {0, 1, 3, 4, 6}
assert decode_to_set(0) == set()
assert not decode_to_set(0) == {}

In [ ]:
# Now for some random number tests. 
# You can execute this cell multiple times... 
from random import randint

print("Integer n   n decoded as set D                D recoded as an integer") 
print("=========   ==================                =======================")
for i in range(5): 
    n = randint(0,5000) 
    D = decode_to_set(n)
    new_n = encode_set(D)
    print("{:<12}{:<34}{:<23}".format(n,str(D),new_n))

## Appendix: finite sets 

Finite sets of integers (or any "immutable" type of python object) can be defined using the `set` class. Below are the basic operations of this class. Brief descriptions are given in the comments. 

In [ ]:
s = set()             # Define s as a new empty set

t = {0,3,6,7,12}      # Directly define t

s.add(n)              # Add n to s (if not already in s)

n in s                # Is n in s?

len(s)                # The cardinality of s

for n in t:           # Iterate over the numbers in s
    print(n,end=" ")  # This is the print out below       
    
s.intersection(t)     # The intersection of sets s and t

s.union(t)            # The union of sets s and t

s.issubset(t)         # Is s a subset of t? 

**Example** Lets play with the sets $\{0,3,6,7,12\}$ and $\{0,1,3,4,6\}$ from above. We firstly show two different methods of defining these sets. You should execute all of the following cells. 

In [ ]:
s = set() 
for u in [0,3,6,7,12]: 
    s.add(u)
print(s)

In [ ]:
t = {0,1,3,4,6}
print(t)

In fact you can also use lists or tuples directly when you define sets.

In [ ]:
s_1 = set([0,3,6,7,12,6,0])  # Notice the repeated numbers in the list
print(s_1)

In [ ]:
s_2 = set((0,3,6,7,12,6,0))   
print(s_2)

In [ ]:
3 in s

In [ ]:
1 in s

In [ ]:
len(s)

In [ ]:
s.remove(7)
print(s)

In [ ]:
# Trying to remove an element that is not in the set throws an exception
try: 
    s.remove(5)     
except KeyError as error: 
    print("This gives the following error message:")
    print("KeyError: 5")

In [ ]:
for n in s: 
    print("{:<2} squared is {}".format(n,n**2))

In [ ]:
v = s.intersection(t)
print(v)

In [ ]:
u = s.union(t)
print(u)

In [ ]:
v.issubset(s)    # Yes the intersection of s and t is a subset of t

In [ ]:
u.issubset(s)   # This would only be the case if t was a subset of s!

**Warning 1.** You may think that `{}` will also define the empty set in python. However this is not the case. In fact `{}` is an empty dictionary. 

In [ ]:
d = {}
type(d)

In [ ]:
try: 
    d.add(5)         
except AttributeError as error: 
    print("This gives the following error message:")
    print("AttributeError: 'dict' object has no attribute 'add'")

**Warning 2.** You saw above that the `set` operator recasts both lists and tuples as sets. However the following code throws an exception. 

In [ ]:
try: 
    c = set(1,2,3,4)
except TypeError as error: 
    print("This gives the following error message:")
    print("TypeError: set expected at most 1 arguments, got 4")